In [ ]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH01 to sweet spot
DC_ss = -0.158888
QDevil.setValue("CH01 Voltage", DC_ss)

# Set other qubits at the antinode
QDevil.setValue("CH02 Voltage", 0.6)
QDevil.setValue("CH03 Voltage", 2.9)
QDevil.setValue("CH04 Voltage", 3.4)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 32
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
freq_TWPA = 6326E6;
pwr_TWPA = -10;
SG.setValue('Frequency', freq_TWPA)
SG.setValue('Power', pwr_TWPA)

client.close()

In [ ]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from qm import SimulationConfig, LoopbackInterface
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
from qualang_tools.bakery import baking
import qdac as qdac
from scipy.io import savemat
from scipy.io import loadmat
from scipy.optimize import curve_fit
from qutip import *
import datetime 
import os
import time


qubit_index = 0 # for UA Q1

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)

## No baking



In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'Zdelay'
qubit_name = 'UA_Q1'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

# The QUA program #
n_avg = 10_000
cooldown_time = 20_000 # ns, set in wait() function
taus = np.arange(4, 152 + 0.1, 1) # Z delay 
qubit_index = 0 
dc = 0.5

with program() as zdelay:
    n = declare(int)
    n_st = declare_stream()
    t = declare(int)
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        
        with for_(*from_array(t, taus)):
            align()
            wait(25, machine.qubits[qubit_index].name, machine.flux_lines[qubit_index].name,
                 machine.resonators[qubit_index].name )

            wait(t, machine.flux_lines[qubit_index].name) # when inside the for loop, t has a delay of an extra 16 ns
            set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
            wait(300 * u.ns, machine.flux_lines[qubit_index].name)
            set_dc_offset(machine.flux_lines[qubit_index].name, "single", machine.flux_lines[qubit_index].max_frequency_point)
            wait(280 * u.ns, machine.qubits[qubit_index].name) 
            # Need to add 100 ns instead of 120 ns because there seems to be a delay of 20 ns between flux line and XY pulse (is it initial tau?)
            play("cw" * amp(0.9), machine.qubits[qubit_index].name, duration = 80*u.ns )
#             wait(10)
            wait(152 - t,machine.qubits[qubit_index].name, machine.flux_lines[qubit_index].name )
            align()
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
    
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time, machine.resonators[qubit_index].name)

        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(taus)).average().save("I")
        Q_st.buffer(len(taus)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulate = False
if simulate:
    simulation_config = SimulationConfig(duration=4000)  # in clock cycles
    job = qmm.simulate(config, zdelay, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(zdelay)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())

In [ ]:
I, Q, iteration = results.fetch_all()
I = u.demod2volts(I, machine.resonators[qubit_index].readout_pulse_length)
Q = u.demod2volts(Q, machine.resonators[qubit_index].readout_pulse_length)
sigs = np.sqrt(I**2 + Q**2)
phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))

fig = plt.figure(figsize=[8, 4])
plt.plot(taus * 4, sigs, 'r')
plt.xlabel("tau [ns]")
plt.ylabel("$\sqrt{I^2 + Q^2}$ [V]")

In [ ]:
savemat(os.path.join(tPath, file_name), {"Tau": taus, "sig": sigs, "sig_phase": phase, "DC_flux": DC_ss, "ROI": ROI, "ROO": ROO, "freq_TWPA": freq_TWPA, "pwr_TWPA": pwr_TWPA})
machine._save(os.path.join(tPath, json_name), flat_data=False)

In [ ]:
%matplotlib qt

## With baking


In [ ]:
now = datetime.datetime.now()
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")

exp_name = 'Zdelay'
qubit_name = 'UA_Q1'

f_str = qubit_name + '_' + exp_name + '_' + month + day + '-' + hour + minute
file_name= f_str+'.mat'
json_name = f_str + '_state.json'

# Set-up the machine and get the config #
machine = QuAM("quam_state.json")
config = build_config(machine)

qubit_index = 0 
n_avg = 10
cooldown_time = 20_000 # ns, set in wait() function

dt = 1
t_vec = np.arange(0, 20, dt) # this number is a test, i need like 500
t_len = len(t_vec)
sample_rate = round(1 / dt * 1e9)

shifted_flux_pulse_list = []

for i in range(t_len):  # Create the different baked sequences
    with baking(config, sampling_rate=sample_rate, padding_method="right") as b:
        b.wait(round(t_vec[i] * (sample_rate / 1e9)), machine.flux_lines[qubit_index].name)
        b.play("const", machine.flux_lines[qubit_index].name)
        b.wait(machine.flux_lines[qubit_index].flux_pulse_length, machine.qubits[qubit_index].name)
        b.play("x180",machine.qubits[qubit_index].name)
#         b.play("cw" * amp(0.9), machine.qubits[qubit_index].name, duration = 80*u.ns ) # need this one
        b.wait(round((np.max(t_vec)+dt - t_vec[i]) * (sample_rate / 1e9)), machine.flux_lines[qubit_index].name)
        print(round((np.max(t_vec)+dt - t_vec[i]) * (sample_rate / 1e9)))
    shifted_flux_pulse_list.append(b)

# The QUA program #

dc = 0.25

with program() as zdelay:
    n = declare(int)
    n_st = declare_stream()
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    segment = declase(int)

    with for_(n, 0, n < n_avg, n + 1):
        with for_(segment, 0, segment <= t_len, segment + 1):
            wait(25, machine.qubits[qubit_index].name, machine.flux_lines[qubit_index].name, 
                    machine.resonators[qubit_index].name ) # wait initial 100 ns

            with switch_(segment):
                for j in range(0, t_len + 1):
                    with case_(j):
                        shifted_flux_pulse_list[j].run(amp_array=[(machine.flux_lines[qubit_index].name, a)])
                            
        for j in range(t_len + 1):
            wait(25, machine.qubits[qubit_index].name, machine.flux_lines[qubit_index].name, 
                    machine.resonators[qubit_index].name ) # wait initial 100 ns
            baking_list[j].run(amp_array = [(machine.qubits[qubit_index].name, 0.5),
                                            (machine.flux_lines[qubit_index].name, 0.5)]) 
                
#             wait(100 * u.ns, machine.qubits[qubit_index].name) 
            # Need to add 100 ns instead of 120 ns because there seems to be a delay of 20 ns between flux line and XY pulse
#             play("cw" * amp(0.9), machine.qubits[qubit_index].name, duration = 80*u.ns )
    #             wait(10)
            align()
        
            measure(
                "readout",
                machine.resonators[qubit_index].name,
                None,
                dual_demod.full("rotated_cos", "out1", "rotated_sin", "out2", I),
                dual_demod.full("rotated_minus_sin", "out1", "rotated_cos", "out2", Q),
            )
            
            save(I, I_st)
            save(Q, Q_st)
            wait(cooldown_time * u.ns)
                
                
                
#         with for_(*from_array(t, taus)):


#             wait(t, machine.flux_lines[qubit_index].name) # when inside the for loop, t has a delay of an extra 16 ns
#             set_dc_offset(machine.flux_lines[qubit_index].name, "single", dc)
#             wait(120 * u.ns, machine.flux_lines[qubit_index].name)
#             set_dc_offset(machine.flux_lines[qubit_index].name, "single", machine.flux_lines[qubit_index].max_frequency_point)
#             wait(100 * u.ns, machine.qubits[qubit_index].name) 
#             # Need to add 100 ns instead of 120 ns because there seems to be a delay of 20 ns between flux line and XY pulse
#             play("cw" * amp(0.9), machine.qubits[qubit_index].name, duration = 80*u.ns )
# #             wait(10)
#             wait(152 - t,machine.qubits[qubit_index].name, machine.flux_lines[qubit_index].name )
#             align()
#             measure(
#                 "readout",
#                 machine.resonators[qubit_index].name,
#                 None,
#                 dual_demod.full("cos", "out1", "sin", "out2", I),
#                 dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
#             )
    
#             save(I, I_st)
#             save(Q, Q_st)
#             wait(cooldown_time, machine.resonators[qubit_index].name)

        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(taus)).average().save("I")
        Q_st.buffer(len(taus)).average().save("Q")
        n_st.save("iteration")

#  Open Communication with the QOP  #
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

simulate = True
if simulate:
    simulation_config = SimulationConfig(duration=4000)  # in clock cycles
    job = qmm.simulate(config, zdelay, simulation_config)
    job.get_simulated_samples().con1.plot()
else:
    qm = qmm.open_qm(config)
    job = qm.execute(zdelay)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")

    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())